In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import round,col, when, udf, format_number, abs, avg
from pyspark.ml import Pipeline
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.sql.types import DoubleType
import pp_events as pp

In [2]:
spark = SparkSession.builder.appName("evaluation").getOrCreate()

your 131072x1 screen size is bogus. expect trouble
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/12/25 17:33:04 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/12/25 17:33:04 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [3]:
events = spark.read.csv('Data/events.csv', header=True, inferSchema=True,sep=';')

In [4]:
events_shot = pp.preprocessing(events,spark)

Data loaded
Spatial data calculated
Preferred foot calculated
Goal column created


Number of players inside the area calculated
Dummies created
Boolean data converted to integer


In [5]:
# save the preprocessed data
events_shot.write.csv('Datas/events_shot.csv', header=True)

24/12/25 17:34:02 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.
24/12/25 17:34:17 WARN TaskSetManager: Stage 5 contains a task of very large size (1681 KiB). The maximum recommended task size is 1000 KiB.


In [6]:
events_shot = spark.read.csv('Datas/events_shot.csv', header=True, inferSchema=True)

In [7]:
events_shot.printSchema()

root
 |-- id: string (nullable = true)
 |-- player_id: double (nullable = true)
 |-- shot_location_x: double (nullable = true)
 |-- shot_location_y: double (nullable = true)
 |-- distance_to_goal: double (nullable = true)
 |-- shot_angle: double (nullable = true)
 |-- preferred_foot_shot: integer (nullable = true)
 |-- other_pp: integer (nullable = true)
 |-- from_fk: integer (nullable = true)
 |-- from_ti: integer (nullable = true)
 |-- from_corner: integer (nullable = true)
 |-- from_counter: integer (nullable = true)
 |-- from_gk: integer (nullable = true)
 |-- from_keeper: integer (nullable = true)
 |-- from_ko: integer (nullable = true)
 |-- header: integer (nullable = true)
 |-- corner_type: integer (nullable = true)
 |-- fk_type: integer (nullable = true)
 |-- pk_type: integer (nullable = true)
 |-- half_volley_technique: integer (nullable = true)
 |-- volley_technique: integer (nullable = true)
 |-- lob_technique: integer (nullable = true)
 |-- overhead_technique: integer (null

In [8]:
features = ['other_pp','from_fk','from_ti','from_corner','from_counter','from_gk','from_keeper','from_ko',
            'header','corner_type','fk_type','pk_type',
            'half_volley_technique','volley_technique','lob_technique','overhead_technique','backheel_technique','diving_h_technique',
            'distance_to_goal', 'shot_angle', 'preferred_foot_shot', 'under_pressure',
            'shot_aerial_won','shot_first_time','shot_one_on_one','shot_open_goal','shot_follows_dribble','players_inside_area']
target = ['goal']

In [9]:
input_size = len(features)
output_size = 2

In [10]:
from pyspark.ml.feature import VectorAssembler

# Define the assembler
feature_assembler = VectorAssembler(inputCols=features, outputCol="features_vector")

# Transform the dataset
assembled_data = feature_assembler.transform(events_shot)
train_data, test_data = assembled_data.randomSplit([0.8, 0.2], seed=42)

In [11]:
from pyspark.ml.classification import LogisticRegression

# Define the logistic regression model
log_reg = LogisticRegression(featuresCol="features_vector", labelCol="goal", maxIter=10)
# Create the pipeline
pipeline = Pipeline(stages=[log_reg])
# Still rerunning all, fix that
xg_model = pipeline.fit(train_data)
# Make predictions
predictions = xg_model.transform(test_data)

evaluator = BinaryClassificationEvaluator(labelCol="goal", rawPredictionCol="rawPrediction")
roc_auc = evaluator.evaluate(predictions)
print(f"ROC-AUC: {roc_auc}")

24/12/25 17:34:57 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS


ROC-AUC: 0.803146473040351


In [21]:
from pyspark.ml.classification import RandomForestClassifier

# Define the Random Forest classifier
rf = RandomForestClassifier(featuresCol="features_vector", labelCol="goal", numTrees=100)

# Create the pipeline
pipeline = Pipeline(stages=[rf])

# Fit the model
xg_model = pipeline.fit(train_data)

# Make predictions
predictions = xg_model.transform(test_data)

evaluator = BinaryClassificationEvaluator(labelCol="goal", rawPredictionCol="rawPrediction")
roc_auc = evaluator.evaluate(predictions)
print(f"ROC-AUC: {roc_auc}")

ROC-AUC: 0.7836640397242735


In [24]:
predictions.show(20,False)

+------------------------------------+---------+---------------+---------------+----------------+----------+-------------------+--------+-------+-------+-----------+------------+-------+-----------+-------+------+-----------+-------+-------+---------------------+----------------+-------------+------------------+------------------+------------------+--------------+---------------+---------------+---------------+--------------+--------------------+-------------------+-----------------+------------+----+-------------------------------------------------------------------+--------------------------------------+----------------------------------------+----------+
|id                                  |player_id|shot_location_x|shot_location_y|distance_to_goal|shot_angle|preferred_foot_shot|other_pp|from_fk|from_ti|from_corner|from_counter|from_gk|from_keeper|from_ko|header|corner_type|fk_type|pk_type|half_volley_technique|volley_technique|lob_technique|overhead_technique|backheel_technique|di

In [30]:
# predict for the whole dataset to create confusion matrix
predictions = xg_model.transform(assembled_data)

In [20]:
from pyspark.ml.classification import MultilayerPerceptronClassifier

# Define the Multilayer Perceptron classifier
layers = [input_size, 64, 32, 16, output_size]

mlp = MultilayerPerceptronClassifier(featuresCol="features_vector", labelCol="goal", maxIter=100, layers=layers, blockSize=128, seed=1234)

# Create the pipeline
pipeline = Pipeline(stages=[mlp])

# Fit the model
xg_model = pipeline.fit(train_data)

# Make predictions
predictions = xg_model.transform(test_data)

evaluator = BinaryClassificationEvaluator(labelCol="goal", rawPredictionCol="rawPrediction")
roc_auc = evaluator.evaluate(predictions)
print(f"ROC-AUC: {roc_auc}")

ROC-AUC: 0.8105376263520693


In [34]:
#### BEST MODEL SO FAR ####
from pyspark.ml.classification import GBTClassifier

# Define the Gradient-Boosted Tree classifier
gbt = GBTClassifier(featuresCol="features_vector", labelCol="goal", maxIter=60)

# Create the pipeline
pipeline = Pipeline(stages=[gbt])

# Fit the model
xg_model = pipeline.fit(train_data)

# Make predictions
predictions = xg_model.transform(test_data)

evaluator = BinaryClassificationEvaluator(labelCol="goal", rawPredictionCol="rawPrediction")
roc_auc = evaluator.evaluate(predictions)
print(f"ROC-AUC: {roc_auc}")

ROC-AUC: 0.8119740211971771


In [42]:
predictions = predictions.withColumn('sb_prediction', round(col('shot_statsbomb_xg')))

In [43]:
from model_evaluation import ModelEvaluation

In [54]:
me_actual = ModelEvaluation(predictions,'goal','prediction')
me_xg = ModelEvaluation(predictions,'sb_prediction','prediction')
me_sb = ModelEvaluation(predictions,'goal','sb_prediction')

In [55]:
me_actual.get_all_metrics()

{'Accuracy': 0.9026737967914439,
 'Precision': 0.7535816618911175,
 'Recall': 0.19338235294117648,
 'Sensitivity': 0.19338235294117648,
 'Specificity': 0.9920333487725799,
 'F1': 0.30778232884727913,
 'FPR': 0.007966651227420102,
 'FNR': 0.8066176470588236}

In [56]:
me_sb.get_all_metrics()

{'Accuracy': 0.9031674208144796,
 'Precision': 0.7163120567375887,
 'Recall': 0.22279411764705884,
 'Sensitivity': 0.22279411764705884,
 'Specificity': 0.9888837424733673,
 'F1': 0.3398766124509255,
 'FPR': 0.0111162575266327,
 'FNR': 0.7772058823529412}

In [35]:
predictions.show(20,False)

+------------------------------------+---------+---------------+---------------+----------------+----------+-------------------+--------+-------+-------+-----------+------------+-------+-----------+-------+------+-----------+-------+-------+---------------------+----------------+-------------+------------------+------------------+------------------+--------------+---------------+---------------+---------------+--------------+--------------------+-------------------+-----------------+------------+----+-------------------------------------------------------------------+------------------------------------------+-----------------------------------------+----------+
|id                                  |player_id|shot_location_x|shot_location_y|distance_to_goal|shot_angle|preferred_foot_shot|other_pp|from_fk|from_ti|from_corner|from_counter|from_gk|from_keeper|from_ko|header|corner_type|fk_type|pk_type|half_volley_technique|volley_technique|lob_technique|overhead_technique|backheel_techniq

In [14]:
# Define a function to extract the probability of the goal (class 1)
def extract_goal_probability(probability):
    return float(probability[1])

# Register the function as a UDF
extract_goal_probability_udf = udf(extract_goal_probability, DoubleType())

# Create a new column with the goal probability
predictions_with_goal_prob = predictions.withColumn("goal_probability", extract_goal_probability_udf(col("probability")))

# Format the goal_probability to remove scientific notation
predictions_with_goal_prob = predictions_with_goal_prob.withColumn("goal_probability", format_number(col("goal_probability"), 10))

# Add a new column showing the difference between shot_statsbomb_xg and goal_probability
predictions_with_goal_prob = predictions_with_goal_prob.\
    withColumn("difference", format_number(col("shot_statsbomb_xg") - col("goal_probability"),10))

predictions_with_goal_prob = predictions_with_goal_prob.\
    withColumn("difference_abs", format_number(abs(col('difference')),10))

# Show the results
predictions_with_goal_prob.select("shot_statsbomb_xg", "goal_probability", "difference",'difference_abs').show(50,False)

+-----------------+----------------+-------------+--------------+
|shot_statsbomb_xg|goal_probability|difference   |difference_abs|
+-----------------+----------------+-------------+--------------+
|0.10166888       |0.1114529814    |-0.0097841014|0.0097841014  |
|0.026109         |0.0443075330    |-0.0181985330|0.0181985330  |
|0.064399794      |0.1162504310    |-0.0518506370|0.0518506370  |
|0.033228997      |0.0525955728    |-0.0193665758|0.0193665758  |
|0.14392917       |0.2581317831    |-0.1142026131|0.1142026131  |
|0.073996074      |0.0731947701    |0.0008013039 |0.0008013039  |
|0.7835           |0.8055580257    |-0.0220580257|0.0220580257  |
|0.10890295       |0.1164947797    |-0.0075918297|0.0075918297  |
|0.05416238       |0.0487551254    |0.0054072546 |0.0054072546  |
|0.07232883       |0.0674026946    |0.0049261354 |0.0049261354  |
|0.087863006      |0.0467176234    |0.0411453826 |0.0411453826  |
|0.121686585      |0.1491249284    |-0.0274383434|0.0274383434  |
|0.0638635

In [15]:
# average of the difference
avg_diff = predictions_with_goal_prob.select(avg(col("difference"))).collect()[0][0]
abs_avg_diff = predictions_with_goal_prob.select(avg(col("difference_abs"))).collect()[0][0]

print(f"Average difference: {avg_diff}")
print(f"Abs average difference: {abs_avg_diff}")

Average difference: -0.006297632282196625
Abs average difference: 0.03164849157705469


In [16]:
# round the goal_probability to 0 or 1, same for the shot_statsbomb_xg and compare with goal to check the accuracy of the model
# vs the accuracy of the shot_statsbomb_xg model
predictions_with_goal_prob = predictions_with_goal_prob.withColumn("goal_probability_rounded", round(col("goal_probability")))\
    .withColumn("shot_statsbomb_xg_rounded", round(col("shot_statsbomb_xg")))

# Check if the rounded probability matches the actual goal for your model
predictions_with_accuracy = predictions_with_goal_prob.withColumn(
    "model_correct", when(col("goal") == col("goal_probability_rounded"), 1).otherwise(0))\
    .withColumn("statsbomb_correct", when(col("goal") == col("shot_statsbomb_xg_rounded"), 1).otherwise(0))
# Calculate accuracy
accuracy_model = predictions_with_accuracy.selectExpr("avg(model_correct) as model_accuracy").first()["model_accuracy"]
accuracy_statsbomb = predictions_with_accuracy.selectExpr("avg(statsbomb_correct) as statsbomb_accuracy").first()["statsbomb_accuracy"]

print(f"Model Accuracy: {accuracy_model * 100:.2f}%")
print(f"StatsBomb xG Accuracy: {accuracy_statsbomb * 100:.2f}%")

Model Accuracy: 90.27%
StatsBomb xG Accuracy: 90.32%


In [17]:
# Testing matching of models, is out model as good as SB_xg?
predictions_with_accuracy = predictions_with_goal_prob.withColumn(
    "model_to_sbxg", when(col("goal_probability_rounded") == col("shot_statsbomb_xg_rounded"), 1).otherwise(0))

xg_accuracy_model = predictions_with_accuracy.selectExpr("avg(model_to_sbxg) as model_accuracy").first()["model_accuracy"]

print(f"Models matching percentage: {xg_accuracy_model * 100:.2f}%")

Models matching percentage: 98.54%


In [ ]:
# show each feature with its coefficient
coefficients = xg_model.stages[-1].coefficients
features_coefficients = list(zip(features, coefficients))

for feature, coefficient in features_coefficients:
    print(f"{feature}: {coefficient}")

In [19]:
train_predictions = xg_model.transform(train_data)
test_predictions = xg_model.transform(test_data)

In [20]:
# calculate accuracy for each model
train_accuracy = train_predictions.filter(train_predictions.goal == train_predictions.prediction).count() / train_predictions.count()

test_accuracy = test_predictions.filter(test_predictions.goal == test_predictions.prediction).count() / test_predictions.count()

print(f"Train accuracy: {train_accuracy * 100:.2f}%")
print(f"Test accuracy: {test_accuracy * 100:.2f}%")

Train accuracy: 91.05%
Test accuracy: 90.27%


In [75]:
spark.stop()